# 1. Basic of Pytorch

### (1) Basic

In [1]:
import torch

- tensor

In [2]:
x = torch.Tensor([[1,2],[3,4]])
x

tensor([[1., 2.],
        [3., 4.]])

- autograd

In [3]:
x = torch.FloatTensor(2,2)
y = torch.FloatTensor(2,2)

In [4]:
y.requires_grad_(True)

tensor([[2.8026e-45, 0.0000e+00],
        [2.8026e-45, 0.0000e+00]], requires_grad=True)

In [5]:
z = (x+y) + torch.FloatTensor(2,2)

- feed forward

In [6]:
def linear(x,w,b):
    y = torch.mm(x,w) + b # mm : matrix multiplication
    return y

In [7]:
x = torch.FloatTensor(16,10)
w = torch.FloatTensor(10,5)
b= torch.FloatTensor(5)
y = linear(x,w,b)

### (2) nn.Module

- nn.Module : 필요한 모델 구조 구현

In [8]:
import torch.nn as nn

In [9]:
class MyLinear(nn.Module):
    def __init__(self,input_size,output_size):
        super().__init__()
        self.w = torch.FloatTensor(input_size,output_size)
        self.b = torch.FloatTensor(output_size)
        
    def forward(self,x):
        y = torch.mm(x,self.w)+self.b
        return y

In [10]:
x = torch.FloatTensor(10,6)
linear = MyLinear(6,2)
y = linear(x)

In [11]:
y

tensor([[-8.6612e+32,  4.5915e-41],
        [-8.6612e+32,  4.5915e-41],
        [-8.6612e+32,  4.5915e-41],
        [-8.6612e+32,  4.5915e-41],
        [-8.6612e+32,  4.5915e-41],
        [-8.6612e+32,  4.5915e-41],
        [-8.6612e+32,  4.5915e-41],
        [-8.6612e+32,  4.5915e-41],
        [-8.6612e+32,  4.5915e-41],
        [-8.6612e+32,  4.5915e-41]])

- simpler

In [13]:
class MyLinear(nn.Module):
    def __init__(self, input_size, output_size):
        super(MyLinear,self).__init__()
        self.linear = nn.Linear(input_size,output_size)
        
    def forward(self,x):
        y = self.linear(x)
        return y

### (3) example

In [14]:
import random
import torch
import torch.nn as nn

- 1. Make a model

In [15]:
class MyModel(nn.Module):
    def __init__(self,input_size,output_size):
        super(MyModel,self).__init__()
        self.linear = nn.Linear(input_size,output_size)
    
    def forward(self,x):
        y = self.linear(x)
        return y

- 2. ground truth

In [16]:
def ground_truth(x):
    return 3*x[:,0] + x[:1] - 2*x[:,2]

- 3. train function

In [24]:
def train(model,x,y,optim):
    optim.zero_grad() # initialize the gradients
    
    y_hat = model(x) # feed-forward
    loss = ((y-y_hat)**2).sum() / x.size(0) # RMSE
    
    loss.backward() # back-propagation
    optim.step()
    
    return loss.data

- 4. set hyperparameters

In [25]:
batch_size = 1
n_epochs = 1000
n_iter = 10000

model = MyModel(3,1)
optim = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.1)

- 5. train the model

In [26]:
for epoch in range(n_epochs):
    avg_loss = 0
    
    for i in range(n_iter):
        x = torch.rand(batch_size,3) # since the input size is 3
        y = ground_truth(x.data)
        loss = train(model,x,y,optim)
        
        avg_loss += loss
        avg_loss = avg_loss / n_iter
    
    x_valid = torch.FloatTensor([[.3,.2,.1]])
    y_valid = ground_truth(x_valid.data)
    
    model.eval()
    y_hat = model(x_valid)
    model.train()
    
    print(avg_loss, y_valid.data[0], y_hat.data[0,0])
    
    if avg_loss < 0.001:
        break

tensor(3.8880e-05) tensor([1.0000, 0.9000, 0.8000]) tensor(1.0188)
